# Cell 1 — Install (only if needed) and imports

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import xgboost as xgb
import shap


pd.options.display.float_format = '{:.3f}'.format
RND = 42
np.random.seed(RND)

## Cell 2 — Load the dataset

In [ ]:

df = pd.read_csv("synthetic_wastewater_3years.csv", parse_dates=["timestamp"])
print("Rows, cols:", df.shape)
df.head(5)

## Cell 2.5 — Define reusability classes based on standard water quality guidelines

  Based on WHO/EPA guidelines for water reuse (adapted for multiclass):
- Drinking: Strictest (BOD<5, COD<10, TSS<1, pH 6.5-8.5, TDS<500)
- Irrigation: Relaxed (BOD<30, COD<100, TSS<50, pH 6-9, TDS<2000)
- Industrial: Moderate (BOD<100, COD<250, TSS<50, pH 6-9, TDS<2000)
- Not_reusable: Fails all

In [ ]:

def classify_reusability(row):
    bod = row['effluent_BOD']
    cod = row['effluent_COD']
    tss = row['effluent_TSS']
    ph = row['effluent_pH']
    tds = row['influent_TDS']  # Use influent_TDS as proxy for effluent TDS

    if bod < 5 and cod < 10 and tss < 1 and 6.5 <= ph <= 8.5 and tds < 500:
        return 'drinking'
    elif bod < 30 and cod < 100 and tss < 50 and 6 <= ph <= 9 and tds < 2000:
        return 'irrigation'
    elif bod < 100 and cod < 250 and tss < 50 and 6 <= ph <= 9 and tds < 2000:
        return 'industrial'
    else:
        return 'not_reusable'

df['reusability'] = df.apply(classify_reusability, axis=1)

print("Reusability distribution:")
print(df['reusability'].value_counts())

## Cell 3 — Quick data inspection
Look for missing values and basic stats.

In [ ]:
df.info()
df.isna().sum()
df.describe().T

## Cell 4 — Visual checks (a few plots)

Plot distributions of key columns and reusability distribution.  
Histogram of influent_BOD and effluent_BOD

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,4))
sns.histplot(df['influent_BOD'], bins=50, kde=True, ax=ax[0]).set_title("influent_BOD")
sns.histplot(df['effluent_BOD'], bins=50, kde=True, ax=ax[1]).set_title("effluent_BOD")
plt.tight_layout()
plt.show()

# Reusability distribution

plt.figure(figsize=(8,4))
df['reusability'].value_counts().plot(kind='bar')
plt.title("Reusability Class Distribution")
plt.show()

## Cell 5 — Basic cleaning function

- Ensure numeric columns exist
- Interpolate small gaps
- Remove duplicates

In [ ]:
def basic_clean(df):
    df = df.copy()
    numeric_cols = [
        "influent_BOD","influent_COD","influent_TSS","influent_pH","influent_TDS",
        "flow_rate","aeration_rate","chemical_dose","sludge_recycle_rate","retention_time","temperature",
        "effluent_BOD","effluent_COD","effluent_TSS","effluent_pH"
    ]
    # Ensure columns exist (create NaNs if missing)
    for c in numeric_cols:
        if c not in df.columns:
            df[c] = np.nan
    df = df.drop_duplicates().sort_values('timestamp').reset_index(drop=True)
    # Interpolate numeric columns for small gaps
    df[numeric_cols] = df[numeric_cols].interpolate(limit_direction='both', axis=0)
    return df

df = basic_clean(df)
df.isnull().sum().loc[lambda x: x>0]

## Cell 6 — Feature engineering

Create rolling means, lag features, normalized dosing, hour/day/month features (for seasonality if needed)

In [ ]:
def feature_engineer(df):
    df = df.copy()
    # rolling 24-hour mean for influent_BOD (assumes hourly index)
    df['influent_BOD_roll24'] = df['influent_BOD'].rolling(window=24, min_periods=1).mean()
    # normalized chemical dose per m3
    df['dose_per_m3'] = df['chemical_dose'] / (df['flow_rate'].replace(0, np.nan))
    df['dose_per_m3'] = df['dose_per_m3'].fillna(0)
    # lag of effluent_BOD (previous hour)
    df['effluent_BOD_lag1'] = df['effluent_BOD'].shift(1).fillna(method='bfill')
    # time features (optional — not used as primary features but useful if needed)
    df['hour'] = df['timestamp'].dt.hour
    df['dayofweek'] = df['timestamp'].dt.dayofweek
    df['month'] = df['timestamp'].dt.month
    return df


df = feature_engineer(df)
df[['timestamp','influent_BOD','influent_BOD_roll24','dose_per_m3','effluent_BOD_lag1','reusability']].head()

## Cell 7 — Prepare features & target

Define features list consistent with production pipeline. We will *not* include timestamp itself as a model feature.

In [ ]:
FEATURES = [
    "flow_rate",
    "influent_BOD",
    "influent_COD",
    "influent_TSS",
    "influent_pH",
    "influent_TDS",
    "aeration_rate",
    "chemical_dose",
    "sludge_recycle_rate",
    "retention_time",
    "temperature",
    "influent_BOD_roll24",
    "dose_per_m3",
    "effluent_BOD_lag1"
]

TARGET = "reusability"

# Ensure all FEATURES exist
for f in FEATURES:
    if f not in df.columns:
        df[f] = np.nan



# Drop rows where TARGET is missing (model needs target)
df_model = df.dropna(subset=[TARGET]).reset_index(drop=True)
print("Rows available for modeling:", len(df_model))

## Cell 8 — Train/test split (time-based)

Use the first 80% as training and last 20% as test (avoid time leakage).

In [ ]:
split_idx = int(len(df_model) * 0.8)
train_df = df_model.iloc[:split_idx].copy()
test_df = df_model.iloc[split_idx:].copy()
print("Train rows:", len(train_df), "Test rows:", len(test_df))

## Cell 9 — Build imputers, scaler, and label encoder (fit on TRAIN only)

We compute medians for imputation and fit StandardScaler on training features. Encode target labels.  
Imputer (median) and scaler

In [ ]:
imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()
label_encoder = LabelEncoder()
X_train_raw = train_df[FEATURES]
X_test_raw = test_df[FEATURES]

# Fit imputer on train
imputer.fit(X_train_raw)
X_train_imp = imputer.transform(X_train_raw)
X_test_imp = imputer.transform(X_test_raw)

# Fit scaler on imputed train
scaler.fit(X_train_imp)
X_train = scaler.transform(X_train_imp)
X_test = scaler.transform(X_test_imp)

# Encode target
y_train_raw = train_df[TARGET]
y_test_raw = test_df[TARGET]
label_encoder.fit(y_train_raw)
y_train = label_encoder.transform(y_train_raw)
y_test = label_encoder.transform(y_test_raw)


print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
print("Classes:", label_encoder.classes_)

## Cell 10 — Train XGBoost classifier with early stopping

We use XGBoost classifier with eval set and early stopping. Adjust hyperparameters as needed.

In [ ]:
model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=RND,
    n_jobs=-1,
    objective='multi:softprob',
    num_class=len(label_encoder.classes_)
)

# Use DMatrix-based early stopping via sklearn API's eval_set parameter
model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=50
)

## Cell 11 — Evaluate model

Compute accuracy, classification report, and confusion matrix.

In [ ]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"Test Accuracy: {accuracy:.3f}")

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title("Confusion Matrix")
plt.show()

## Cell 12 — Feature importance (XGBoost built-in)

In [ ]:
fi = pd.Series(model.feature_importances_, index=FEATURES).sort_values(ascending=False)
plt.figure(figsize=(8,5))
fi.plot(kind='bar')
plt.title("XGBoost feature_importances_")
plt.tight_layout()
plt.show()

## Cell 13 — SHAP explainability (global + one sample)

 Note: SHAP can be resource-intensive. Use a small background sample.  
 Create explainer (TreeExplainer for xgboost)

In [ ]:
explainer = shap.Explainer(model)

# Use a small background dataset (sample from train)
bg = X_train[np.random.choice(len(X_train), size=min(500, len(X_train)), replace=False)]

# Compute SHAP values for a subset of test set to save time
X_test_sample = X_test[:1000]
shap_values = explainer(X_test_sample)

# Global summary plot
shap.summary_plot(shap_values, features=X_test_sample, feature_names=FEATURES, show=True)

## Cell 14 — Save artifacts for production

We save: model.joblib, imputer.joblib, scaler.joblib, features.joblib, label_encoder.joblib, metadata.

In [ ]:
ARTIFACT_DIR = "models_v1"
os.makedirs(ARTIFACT_DIR, exist_ok=True)
joblib.dump(model, os.path.join(ARTIFACT_DIR, "model.joblib"))
joblib.dump(imputer, os.path.join(ARTIFACT_DIR, "imputer.joblib"))
joblib.dump(scaler, os.path.join(ARTIFACT_DIR, "scaler.joblib"))
joblib.dump(FEATURES, os.path.join(ARTIFACT_DIR, "features.joblib"))
joblib.dump(label_encoder, os.path.join(ARTIFACT_DIR, "label_encoder.joblib"))


meta = {
    "accuracy": float(accuracy),
    "train_rows": int(len(train_df)),
    "test_rows": int(len(test_df)),
    "classes": list(label_encoder.classes_),
    "trained_at": pd.Timestamp.now().isoformat()
}

joblib.dump(meta, os.path.join(ARTIFACT_DIR, "meta.joblib"))

print("Saved artifacts to:", ARTIFACT_DIR)

## Cell 15 — Quick function: load artifacts and predict on a pandas DataFrame

This emulates what your API will do: load imputer & scaler, preprocess features (same order), and predict.

In [ ]:
def load_artifacts(artifact_dir):
    model = joblib.load(os.path.join(artifact_dir, "model.joblib"))
    imputer = joblib.load(os.path.join(artifact_dir, "imputer.joblib"))
    scaler = joblib.load(os.path.join(artifact_dir, "scaler.joblib"))
    features = joblib.load(os.path.join(artifact_dir, "features.joblib"))
    label_encoder = joblib.load(os.path.join(artifact_dir, "label_encoder.joblib"))
    return model, imputer, scaler, features, label_encoder

def preprocess_df_for_model(df_in, imputer, scaler, features):
    # expects feature-engineered df (with same columns). We'll create missing features if absent.
    X = df_in[features].copy()
    for c in features:
        if c not in X.columns:
            X[c] = np.nan
    X_imp = imputer.transform(X)
    X_scaled = scaler.transform(X_imp)
    return X_scaled

# Load artifacts
model_l, imputer_l, scaler_l, features_l, le_l = load_artifacts(ARTIFACT_DIR)

# Example: predict first 5 rows from test_df
X_test_proc = preprocess_df_for_model(test_df, imputer_l, scaler_l, features_l)
preds_encoded = model_l.predict(X_test_proc[:5])
preds_labels = le_l.inverse_transform(preds_encoded)
print("Preds:", preds_labels)
print("Actuals:", test_df[TARGET].values[:5])

## Cell 16 — (Optional) Save a small sample CSV with predictions

Useful for demo or Streamlit preview.

In [ ]:
out = test_df.copy().head(1000)
out_proc = preprocess_df_for_model(out, imputer_l, scaler_l, features_l)
out['reusability_pred'] = le_l.inverse_transform(model_l.predict(out_proc))
out[['timestamp','effluent_BOD','effluent_COD','effluent_TSS','effluent_pH','reusability','reusability_pred']].head()
out.to_csv("predictions_sample.csv", index=False)
print("Saved sample predictions to predictions_sample.csv")

## Cell 17 — (Optional) FastAPI snippet to serve predictions

Run separately as `src/api.py`. This is sample code to show how to load artifacts and serve endpoints.

In [ ]:
api_code = """
from fastapi import FastAPI, UploadFile, File, HTTPException
import pandas as pd, io, joblib, os
from pipeline import feature_engineer  # if you put FE logic in pipeline
app = FastAPI()
ARTIFACT_DIR = "models_v1"
model = joblib.load(os.path.join(ARTIFACT_DIR, "model.joblib"))
imputer = joblib.load(os.path.join(ARTIFACT_DIR, "imputer.joblib"))
scaler = joblib.load(os.path.join(ARTIFACT_DIR, "scaler.joblib"))
features = joblib.load(os.path.join(ARTIFACT_DIR, "features.joblib"))
label_encoder = joblib.load(os.path.join(ARTIFACT_DIR, "label_encoder.joblib"))

@app.post("/predict_csv")
async def predict_csv(file: UploadFile = File(...)):
    if not file.filename.endswith(".csv"):
        raise HTTPException(status_code=400, detail="CSV only")
    data = await file.read()
    df = pd.read_csv(io.BytesIO(data), parse_dates=['timestamp'])
    df = feature_engineer(df)  # apply the same FE you used in notebook
    X = df[features].copy()
    for c in features:
        if c not in X.columns:
            X[c] = np.nan
    X_imp = imputer.transform(X)
    X_scaled = scaler.transform(X_imp)
    preds_encoded = model.predict(X_scaled)
    preds_labels = label_encoder.inverse_transform(preds_encoded)
    df['reusability_pred'] = preds_labels
    return {"rows": len(df), "preview": df.head(10).to_dict(orient='records')}
"""

print(api_code[:1000])

## Final notes & next steps

- The model now predicts reusability classes: 'not_reusable', 'industrial', 'irrigation', 'drinking'.
- Based on effluent quality parameters.
- Run the notebook cells to train and generate the saved artifacts in `models_v1`.
- Ask me to produce the API files or repo scaffolding next.